# Transient object detection

## 1. Introduction

This notebook walks through how to develop a workflow for generating synthetic data and training a machine learning model to detect meteors from images.

This tutorial has been prepared by:
- [Andrew Rohl](http://computation.curtin.edu.au/about/steering-committee/director/)
- [Shiv Meka](http://computation.curtin.edu.au/about/computational-specialists/humanities/)
- [Kevin Chai](http://computation.curtin.edu.au/about/computational-specialists/health-sciences/)

from the [Curtin Institute for Computation](http://computation.curtin.edu.au) at Curtin University in Perth, Australia for the [7th International Conference on Smart Computing & Communications (ICSCC 2019)](http://icscc.online/) hosted at Curtin University in Miri, Sarawak, Malaysia on the 28-30 June 2019.

## 2. Project

### Background

The Curtin Institute for Computation was asked to develop a machine learning model to detect meteor trails (fireballs) by researchers from the [Curtin Institute for Radio Astronomy](http://astronomy.curtin.edu.au/) ([ICRAR](https://www.icrar.org/) - Curtin University node) from images of the night sky captured in the Australian desert by the [Desert Fireball Network](http://fireballsinthesky.com.au/). 

![Fireballs](assets/desert_fireball_network.png)

<div style="text-align:center;font-weight:bold">Figure: Desert Fireball Network camera locations</div>

The objective of the project is to find these meteors in optical images and to compare against the radio emissions recorded at the same time and location by the [Murchison Widefield Array](http://www.mwatelescope.org).

### Team

*Curtin Institute for Radio Astronomy (CIRA)*
* Paul Hancock
* Xiang Zhang
* Sean Mattingley
* Steven Tingay
* Randall Wayth

*Desert Fireball Network*
* Hadrien Devillepoix
* Phil Bland

*Curtin Institute for Computation*
* Shiv Meka
* Kevin Chai

## 3. Data preparation

### Dataset

The dataset is comprised of images taken during the Geminid meteor shower on the 14th of December 2015. Images were captured from a camera based at a station and a camera that was set up on the back of a truck (mobile) to capture images from multiple locations. The images were reviewed and annotated by astronomers to construct the dataset shown in Table 1.

<p style="text-align:center;font-weight:bold">Table 1: Dataset</p>

| Camera      | Images    | Meteors
|:------------|:----------|:--------|
| mobile      | 1,561     | 48      |
| station     | 1,330     | 24      |

The raw dataset contains RGB images with a resolution of 7360x4912 pixels (~6.3MB each). Meteors within these images were manually identified by astronomers within the project team. Examples of these meteors are shown below.

![Fireballs](assets/meteors.png)

<div style="text-align:center;font-weight:bold">Figure: Cropped images containing meteors in the dataset</div>

From preliminary experiments, we observed that meteors could be identified without colour and using lower resolution images. The benefit of compressing the images is that it reduces the computational overhead required and therefore allows us to train a meteor detection model faster. 

The raw images are transformed into grayscale and to a resolution of 1840x1228, resulting in ~240KB for each image. An example is shown below.

![Night Sky](assets/night_sky_example.jpg)

<div style="text-align:center;font-weight:bold">Figure: Transformed night sky image example</div>

This data preparation pipeline can performed using the Python Imaging Library (PIL / Pillow) module. An example is presented below.

In [ ]:
%matplotlib inline
import math
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras.backend as K
from PIL import Image, ImageDraw
from workshop import *

img = Image.open('assets/data_prep_example.jpg')

# Resize the new image to 50% of the original
img_new = img.resize((int(img.width * 0.5), int(img.height * 0.5)), Image.ANTIALIAS)

# Convert to grayscale
img_new = np.array(img_new.convert('L'))

# Plot the original and transformed image
# Original
fig = plt.figure(figsize=(7, 7))
fig.gca().title.set_text('Original')
fig.gca().imshow(img);

# resized, grayscale
fig = plt.figure(figsize=(3.5, 3.5))
fig.gca().title.set_text('50% resized, grayscale')
fig.gca().imshow(img_new, cmap='gray');

### Synthetic data

The dataset does not contain enough meteors to train a robust and accurate machine learning model. The research team simplified the problem to identify any transient objects (streaks) in the images. Transient objects can be meteors, satellites and planes.

Therefore, we experimented with generating a synthetic dataset of transient objects to train our model. We manually identified images of the night sky that contained no streaks and used these as the background to generate our synthetic / fake meteor images.

Let's inspect two background examples.

In [ ]:
# Plot the original and transformed image
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 16))
img1 = Image.open('assets/background1.jpg')
ax1.imshow(np.asarray(img1), cmap='gray');
ax1.set_title('Background images')
img2 = Image.open('assets/background2.jpg')
ax2.imshow(np.asarray(img2), cmap='gray');

Note: the second image is more illuminated than the first with less visible stars. This is because the second image was taken closer to dawn / sunrise. We use 272 different background images to ensure enough variation (different real world conditions) is added to our dataset.

Define functions to create tiled images (200x200 pixel resolution).

In [ ]:
# Computes distance between two points in pixels
def dist(A): 
    return(((A[0][0] - A[0][2])**2 + (A[0][1] - A[0][3])**2)**0.5)

def generate_streaks(seed=0):
    np.random.seed(seed)
    tile_w = 200 
    tile_h = 200
    n=10
    images = []

    for __ in range(n):
        bg_rand = np.random.choice([1, 2])    
        img = Image.open('assets/background' + str(bg_rand) + '.jpg')
        width, height = img.size
        marker_x = np.random.randint(0, width - tile_w)
        marker_y = np.random.randint(0, height - tile_h)

        # crop a portion of the full image
        im = img.crop((marker_x, marker_y, marker_x + tile_w, marker_y + tile_h))
        draw = ImageDraw.Draw(im)

        A = [(np.random.rand() * im.size[0], np.random.rand() * im.size[1],
              np.random.rand() * im.size[0], np.random.rand() * im.size[1])]

        # The lines shouldn't be too short or too long
        while dist(A) <= 30 or dist(A) >= 300:
            A = [(np.random.rand() * im.size[0], np.random.rand() * im.size[1],
                  np.random.rand() * im.size[0], np.random.rand() * im.size[1])]

        if dist(A) > 30 and dist(A) < 300: 
            # Brightness should be over certain threshold, 
            # lower the brightness -> harder to train, more resilient
            color_rnd = np.random.uniform(0.8, 1.0)  
            width_rand = np.random.choice([1, 2])
            draw.line([A[0][0], A[0][1], A[0][2], A[0][3]], fill=int(color_rnd * 250), width=width_rand)
            del draw
        
        images.append(im)
        
    plot_streaks(images)

def plot_streaks(images):
    plt.figure(figsize=(20,10))
    columns = 5
    for i, image in enumerate(images):
        ax = plt.subplot(len(images) / columns + 1, columns, i + 1)
        ax.imshow(np.asarray(image), cmap='gray')
        ax.axis('off')

Let's look at some of the generated images

In [ ]:
interact(generate_streaks, 
         seed=widgets.IntSlider(min=0, max=10, step=1, continuous_update=False));

Currently our function only generates image tiles with streaks. Now let's define a more complete function that will generate image with/without streaks and assign the correct label (0 = no streak, 1 = streak) to the image. 

In [ ]:
def fake_image(file_name="meteors.hd5",batch_size=80,seed=3):
    '''
    file_name: hdf file with images of night_sky and validation
    batch_size: images per step
    '''
    filter_w=200 #output image width
    filter_h=200 #output image height
    np.random.seed(seed)
    def dist(A): #Computes distance between two points in px
        return(((A[0][0]-A[0][2])**2+(A[0][1]-A[0][3])**2)**0.5)

    count=0
    hdf=h5py.File("meteors.hd5","r")
    bg_imgs=hdf["night_sky"][:]
    while True:
        out_images=[] #list to store
        out_preds=[]
        for i in range(batch_size):
            myrand=np.random.randint(bg_imgs.shape[0]) #Open files with random background and w/o meteorites
            im=Image.fromarray(bg_imgs[myrand]) #im stores images with random background
            width,height=im.size
            marker_x=np.random.randint(0,width-filter_w)
            marker_y=np.random.randint(0,width-filter_h)
            #crop a portion of the image
            im=im.crop((marker_x,marker_y,marker_x+filter_w,marker_y+filter_h))
            draw = ImageDraw.Draw(im)
            A=[(np.random.rand()*im.size[0], np.random.rand()*im.size[1],
                np.random.rand()*im.size[0], np.random.rand()*im.size[1])]
            myrand=str(myrand)
            flag_present=False
            if (np.random.rand()>0.5): #Statistically, only half the samples would have meteorites
                if ((dist(A)>50) and (dist(A)<400)): #The lines shouldn't be too short or too long
                    color_rnd=np.random.uniform(0.5,0.8)  #Brightness should be over certain threshold, lower the brightness -> harder to train, more resilient
                    width_rand=np.random.choice([1,2])
                    draw.line([A[0][0],A[0][1],A[0][2],A[0][3]], fill=int(color_rnd*100),width=width_rand)
                    flag_present=True
                    del draw
            out_images.append(np.array(im).reshape([filter_w,filter_h,1]))
            out_preds.append(flag_present*1)
        yield np.array(out_images),np.array(out_preds)

In [ ]:
!ls

In [ ]:
#Visualize images
out = next(synthetic_images(batch_size=1))
print('Label: %d' % (out[1][0]))
Image.fromarray(out[0][0].squeeze())

## 4. Model

Through our experimentation, we designed a CNN model architecture that was able to achieve good classification performance.

In [ ]:
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization
from keras.models import Model
import os
import random
import glob
import numpy as np

def create_model(seed):
    np.random.seed(seed)
    random.seed(seed)
    # Input image tiles are 200 x 200 x 1 images
    inp = Input(shape=(200, 200, 1))

#     # 3 sets of convolution + pooling layers, REctified Linear Unit activation (non linear)
#     x = Conv2D(8, 3, activation='relu')(inp)
#     x = MaxPool2D()(x)
#     x = BatchNormalization()(x)
#     x = Conv2D(5, 3, activation='relu')(x)
#     x = MaxPool2D()(x)
#     x = BatchNormalization()(x)
#     x = Conv2D(4, 3, activation='relu')(x)
#     x = MaxPool2D()(x)
#     x = BatchNormalization()(x)
#     x = Flatten()(x)



    # 5 fully connected layers
    for __ in range(3):
        x = Dense(5)(x)

    out = Dense(1, activation='sigmoid')(x)

    model = Model(inp, out)
    model.compile(optimizer='adagrad', loss='binary_crossentropy')
    
    return model

seed = 1
model = create_model(seed)
model.summary()

## 5. Training

Let's train our CNN model using the `fit_generator` method. This makes use of our synthetic_images function that generates our streaks and no streaks images on the fly.

In [ ]:
files = sorted(glob.glob('meteors_validation/*/*jpg'))

data = []
labels = []

for index, file in enumerate(files):
    img = Image.open(file)
    data.append(np.array(img))
    if index < 4:
        labels.append(0)
    else:
        labels.append(1)
        
labels = np.array(labels)
data = np.array(data).reshape(len(labels), 200, 200, 1)

In [169]:
# ~2 mins to train 
#model = create_model(2)
seed=100
np.random.seed(seed)
random.seed(seed)
input_layer=Input(shape=(200,200,1)) #1 - channel implies just grayscale
output_layer=[]
for i in range(4):
    output_layer=Conv2D(12+(2*i),3,activation='relu')((output_layer,input_layer)[i==0])
    output_layer=MaxPool2D(2)(output_layer)

output_layer=Flatten()(output_layer)

for i in range(5):
    output_layer=Dense(5)(output_layer)
output_layer=Dense(1,activation='sigmoid')(output_layer)

model=Model(input_layer,output_layer)
model.compile(optimizer='adagrad',loss='binary_crossentropy')


model.fit_generator(fake_image(batch_size=15,seed=seed), validation_data=(data,labels),
                    steps_per_epoch=30, epochs=10)

Epoch 1/10
30/30 [==============================] - 9s 315ms/step - loss: 0.7024 - val_loss: 0.7038
Epoch 2/10
30/30 [==============================] - 10s 319ms/step - loss: 0.6345 - val_loss: 0.7098
Epoch 3/10
30/30 [==============================] - 10s 329ms/step - loss: 0.5293 - val_loss: 0.6063
Epoch 4/10
30/30 [==============================] - 14s 453ms/step - loss: 0.5124 - val_loss: 0.4998
Epoch 5/10
30/30 [==============================] - 11s 356ms/step - loss: 0.4434 - val_loss: 0.5540
Epoch 6/10
30/30 [==============================] - 13s 426ms/step - loss: 0.3919 - val_loss: 0.4294
Epoch 7/10
30/30 [==============================] - 14s 471ms/step - loss: 0.3149 - val_loss: 0.4230
Epoch 8/10
30/30 [==============================] - 11s 383ms/step - loss: 0.3091 - val_loss: 0.4250
Epoch 9/10
30/30 [==============================] - 12s 402ms/step - loss: 0.2003 - val_loss: 0.1880
Epoch 10/10
30/30 [==============================] - 11s 372ms/step - loss: 0.1776 - val_los

## 6. Validation

Run the trained model on unseen validation images

In [170]:
# TODO: show some samples of background / meteors
import glob
@interact
def showimage(filename=sorted(glob.glob('meteors_validation/*/*jpg'))):
    img=Image.open(filename)
    img_arr=np.array(img)
    print ("Prediction: ", np.round(model.predict(img_arr.reshape([1,
                                                                  img_arr.shape[0],
                                                                  img_arr.shape[1],1]))))
    return img

interactive(children=(Dropdown(description='filename', options=('meteors_validation/0/1_0.jpg', 'meteors_valid…

We've only evaluated the model on a sample of 8 validation images. How well does our model perform on the validation and test datasets? In our real experimentation, we needed to take a more involved  approach to train an accurate model:

<p><center><img src='assets/training.png'></center></p>

Initially we generated a dataset with bright streaks and progressively made them fainter and fainter. 

<p><center><img src='assets/results.png'></center></p>

## 7. Visualisation

Let's visualise the convolutional layers of the trained model. This allows us to check our model to determine what pixels excite / activate our convolution filters and allows us to debug errors / difficult images (e.g. detecting very faint meteors on well-lit images of the night sky).





In [ ]:
# Visualise the layer
def vis_layer(v, ix, iy, ch, cy, cx, p=0) :
    v = np.reshape(v, (iy, ix, ch))
    ix += 2
    iy += 2
    npad = ((1, 1), (1, 1), (0, 0))
    v = np.pad(v, pad_width=npad, mode='constant', constant_values=p)
    v = np.reshape(v, (iy, ix, cy, cx)) 
    v = np.transpose(v, (2,0,3,1)) #cy,iy,cx,ix
    v = np.reshape(v, (cy*iy, cx*ix))
    return v

def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output])
    activations = get_activations([X_batch, 0])
    return activations

def plot_layer(feature_map, width, height, channels, rows, columns):
    v  = vis_layer(feature_map, width, height, channels, rows, columns)
    plt.figure(figsize = (16, 16))
    plt.imshow(v, interpolation='nearest', cmap='gray')    
    plt.axis('off');
    
def plot_activations(image):
    img = Image.open(image)
    img = np.array(img).reshape(1, 200, 200, 1)

    conv_layers = [i for i, layer in enumerate(model.layers) if type(layer) == Conv2D]
    print('Conv layers 1-%d' % (len(conv_layers)))
    
    for index, conv in enumerate(conv_layers):
        feature_map = get_activations(model, conv, img)
        image_shape = feature_map[0].shape[1]
        n_filters = model.layers[conv].get_weights()[0].shape[3]
        plot_layer(feature_map, image_shape, image_shape, n_filters, 2, int(n_filters / 2))

files = sorted(glob.glob('meteors_validation/1/*jpg'))
_ = interact(plot_activations, image=files)

This visualisation gives us an insight to what the model is paying attention to. Ideally, we want the filters to activate for the streaks (i.e. pixels) and ignore / remove (despeckle) the stars from the image background.

## 8. Test dataset

Now we have trained an accurate model, we need to script a workflow that takes the original images and generate the detections. This process is depicted below:

<p><center><img src='assets/detection.png'></center></p>

The workflow scripts are quite involved and are not presented here. However, here is an example of the visualised detections for a given image.

<p><center><img src='assets/test.jpg'></center></p>

### Download data

Andrew to download meteors.md5 file from us before he leaves

In [ ]:
#Download and uncompress the dataset. 
#Does nothing if the file already exists
# Download("https://cloudstor.aarnet.edu.au/plus/s/EubVHMQ1lq7zOfD/download","workshop.tar.gz")


## 9. Conclusion

Andrew to wrap up about what was covered in the tutorial